In [ ]:
import os
import sys
import time as t_util
import numpy as np
import pandas as pd
import yaml
import cftime
import xarray as xr


In [ ]:
#Read main paths
with open('../path_main.txt', 'r') as file:   path_main = file.read()
with open('../path_CMIP6.txt', 'r') as file:  path_cmip6 = file.read()
    
dir_CMIP6   = path_cmip6
dir_scripts = f'{path_main}Scripts/'
dir_names   = f'{path_main}Scripts/Model_lists/'
dir_EMT     = f'{path_main}Data/CMIP6/EMT/'
dir_Yearmax = f'{path_main}Data/CMIP6/Yearmax/'
if not os.path.exists(dir_Yearmax):  os.mkdir(dir_Yearmax)


## Prepare variables and parameters

In [8]:
#Define models and SSPs which should be used
all_models = dict()
all_models['ssp585'] = []
with open(dir_names + 'Models_CMIP6_SSP585.txt', 'r') as filehandle:
    for line in filehandle:
        all_models['ssp585'].append(line[:-1])

#Define variables
variables = ['tasmax']#, 'tasmin']

#Define output variable names
vars_out = dict()
vars_out['tasmax'] = 'TXx'
vars_out['tasmin'] = 'TNx'        
        
#Define scenarios
scenarios = ['historical']#, 'ssp585']

#Define time periods
time_periods = [[1981, 2010]]#, [2070, 2099]]

#Define EMT
dEMT = 3
dEMT_str = 'ESAT-3.0K'


## Calculation for 1981-2010 and 2070-2099

In [ ]:
#Loop over models
for model in all_models['ssp585']:
    
#     if model in ['ACCESS-CM2','ACCESS-ESM1-5','CNRM-CM6-1','CNRM-CM6-1-HR','CNRM-ESM2-1','CanESM5',
#                   'EC-Earth3','EC-Earth3-Veg']:
#         continue
    
    
    print(model, end=': ')
    
    #Select ensemble member
    if model in ['CNRM-CM6-1', 'CNRM-ESM2-1', 'CNRM-CM6-1-HR', 'UKESM1-0-LL', 'MIROC-ES2L']:
        member = "r1i1p1f2"
    elif model in ['HadGEM3-GC31-LL', 'HadGEM3-GC31-MM']:
        member = "r1i1p1f3"
    else:
        member = "r1i1p1f1"            

    #Loop over variables
    for variab in variables:

        print(variab, end=', ')
        
        #Creat output directory
        dir_Ymax_out = dir_Yearmax + vars_out[variab] + '/'
        if not os.path.exists(dir_Ymax_out):  os.mkdir(dir_Ymax_out)          
        
        #Get file names
        fnames_merge = []
        for scen in scenarios:
        
            dir_data = dir_CMIP6 + scen + '/' + variab + '/'
            fnames = [dir_data + file for file in os.listdir(dir_data) if model + '_' in file and member in file and variab in file and scen in file and '_day_' in file]
            if len(fnames)!=1: sys.exit('Filename not uniquely defined')

            #Add to list for all files for merging
            fnames_merge = fnames_merge + fnames

        #Read and merge data
        data = xr.concat((xr.open_dataset(fname) for fname in fnames_merge), dim='time')
        data = data.rename({variab: vars_out[variab]})
        
        #Loop over time periods
        for time_per in time_periods:

            #Select time
            time_sel = slice(str(time_per[0]), str(time_per[1]))
            data_sel = data.sel(time=time_sel)

            #Calculate yearly maximum
            data_Yearmax  = data_sel.resample(time='1Y').max()

            #Create output file names
            output_str = "_" + model + '_' + "-".join(scenarios) +  "_" + time_sel.start + "-" + time_sel.stop + ".nc"
            fname_Yearmax = dir_Ymax_out + vars_out[variab] + "_Yearmax" + output_str

            #Save in file
            if os.path.exists(fname_Yearmax): os.remove(fname_Yearmax)
            data_Yearmax.to_netcdf(fname_Yearmax)

    print('')

ACCESS-CM2: tasmax, ACCESS-ESM1-5: tasmax, CNRM-CM6-1: tasmax, CNRM-CM6-1-HR: tasmax, 

## Calculation for 3°C European warming

In [ ]:
scen = 'ssp585'

#Loop over models
for model in all_models['ssp585']:
    
    print(model, end=': ')

    #Select ensemble member
    if model in ['CNRM-CM6-1', 'CNRM-ESM2-1', 'CNRM-CM6-1-HR', 'UKESM1-0-LL', 'MIROC-ES2L']:
        member = "r1i1p1f2"
    elif model in ['HadGEM3-GC31-LL', 'HadGEM3-GC31-MM']:
        member = "r1i1p1f3"
    else:
        member = "r1i1p1f1"            

    #Read European mean temperature (EMT)
    files_EMT = sorted([dir_EMT + file for file in os.listdir(dir_EMT) if model + '_' in file and 'EMT_' in file])
    data_EMT  = xr.concat((xr.open_dataset(file) for file in files_EMT), dim='time')

    #Calculate EMT relative to 1981-2010 and calculate 20-year means
    dataEMT_ref = data_EMT.sel(time=slice('1981', '2010')).mean('time')
    dataEMT_rel = data_EMT - dataEMT_ref
    dataEMT_20y = dataEMT_rel.rolling(time=20, center=True).mean()    

    #Identify 20-year period in wich level is reached for first time
    ind  = np.where(dataEMT_20y.tas>dEMT)[0]
    if len(np.where(dataEMT_20y.tas>dEMT)[0])==0:
        print(model + " does not reach 3°C European warming")
        continue
    else:
        ind = ind[0]

    #Get extent of 20-year period 
    central_year = dataEMT_20y.isel(time=ind).time.dt.year
    start_year   = int(central_year - 20 / 2)
    end_year     = int(central_year + (20 / 2 - 1))
    years_sel    = slice(str(start_year), str(end_year))        

    #Check that EMT is reached within the read time of the climate models
    if start_year<2020:
        print(model, end=': ')
        print(start_year, end=', ')
    if end_year>2099:
        print(model, end=': ')
        print(end_year, end=', ')
        
    #Loop over variables
    for variab in variables:

        print(variab, end=', ')
        
        #Creat output directory
        dir_Ymax_out = dir_Yearmax + vars_out[variab] + '/'
        if not os.path.exists(dir_Ymax_out):  os.mkdir(dir_Ymax_out)          
        
        #Get file names
        dir_data = dir_CMIP6 + scen + '/' + variab + '/'
        fnames = [dir_data + file for file in os.listdir(dir_data) if model + '_' in file and member in file and variab in file and scen in file and '_day_' in file]
        if len(fnames)!=1:
            fnames = [file for file in fnames if '2300' not in file]
            fnames = [file for file in fnames if '_gn_' in file]
        if len(fnames)!=1:
            sys.exit('Filename not uniquely defined')

        #Read and merge data
        data = xr.concat((xr.open_dataset(fname) for fname in fnames), dim='time')

        #Select time
        data = data.sel(time=years_sel)

        #Calculate yearly maximum
        data_Yearmax = data.resample(time='1Y').max()
        
        #Rename data
        data_Yearmax = data_Yearmax.rename({variab: vars_out[variab]})
        
        #Create output file names
        output_str = "_" + model + '_' + "-".join(scenarios) +  "_" + dEMT_str + ".nc"
        fname_Yearmax = dir_Ymax_out + vars_out[variab] + "_Yearmax" + output_str

        #Save in file
        if os.path.exists(fname_Yearmax): os.remove(fname_Yearmax)
        data_Yearmax.to_netcdf(fname_Yearmax)

    print('')